# Fiona

[Fiona](http://toblerity.org/fiona/) is build on top of OGR and it introduces the object-oriented approach to the work with vector data. The data are re-mapped in to GeoJSON structure. The interaction is more straight-forward compared to OGR.

Fiona is not suited to be the best for all tasks - simplicity is buyed back by the fact, all data need to be loaded in memmory. Also speed can be lower, when dealing with larger amounts of data. Where OGR uses pointers, Fiona has to deal with whole python objects. Generally: if you need to filter vector features, OGR should be faster, if you need to work with features sequentially, Fiona should be better.

**Note**:
> Fiona is great for binary data formats. For JSON, use the `json` python package. For data stored in database, use standard Python database interface.

First we need to create so called collection of features:

In [ ]:
import fiona
chko = fiona.open('../../data/natural.shp', 'r') # collection - data are stored in ../../data directory 
chko

We can now mine some metadata about this feature collection, see [documentation](http://toblerity.org/fiona/manual.html)

In [ ]:
# driver
chko.driver

In [ ]:
# coordinate reference system
chko.crs

In [ ]:
# file name
chko.path

In [ ]:
# layer name
chko.name

In [ ]:
# bounding box coordinates
chko.bounds

In [ ]:
# everything in one step
print(chko.meta)

In [ ]:
# try better format
import json
print(json.dumps(chko.meta, sort_keys=True, indent=4, separators=(',', ': ')))

## Features in collections
Features within opened collection can be iterated

In [ ]:
print(len(chko))

## Coordinate reference systems
Fiona comes with primitive tools for dealing with coordinate reference systems, with GDAL library in the background

In [ ]:
natural = fiona.open('../../data/natural.shp', 'r')
from fiona.crs import to_string
print(to_string(natural.crs))

Simillary, new CRS definition can be created

In [ ]:
from fiona.crs import from_epsg
from_epsg(3857)

## Walking through features
we can either iterate through available features

In [ ]:
for feature in chko[0:10]:
    print(feature['geometry']['type'])

or particular feature can be choosed

In [ ]:
print(chko[54]['properties']['name'])

## Feature geometry and shapely library
[Shapely](http://toblerity.org/shapely) converts feature geometry into GeoJSON structure. It also contains tools for geometry manipulations

In [ ]:
from shapely.geometry import shape
cr = chko[54]
poly = shape(cr['geometry'])
poly.bounds

Now we can either generalize given geometry or create buffer

In [ ]:
simple = poly.simplify(10)
simple.intersects(poly)
buff = poly.buffer(10)
buff.contains(poly)

some feature attributes can be fixed as well, at the end, we write new created feature down

In [ ]:
from shapely.geometry import mapping
import copy
feature = copy.deepcopy(cr)
feature['id'] = -1
feature['geometry'] = mapping(buff)
feature['properties']['name'] = u'Mordor'
chko = fiona.open('../../data/natural.shp', 'a')
print("Features before: %d " % len(chko))
chko.write(feature)
print("Features after: %d " % len(chko))
chko.close()